In [1]:
import src.generation.gamma as ga
import src.generation.multivariate_normal as mn

from src.salp.salp import SALP
from src.salp.salp import Node

from src.algorithms.round_robin import RoundRobin
from src.algorithms.best_fit import BestFit

from src.evaluation.core import pipeline

import numpy as np

In [2]:
# TODO algorithms
# SALP
def salp_wrapper(N: int, load_vectors):
    return SALP(N, load_vectors, False)
# Best Fit
def bf_wrapper(N: int, load_vectors):
    bf = BestFit(N, load_vectors, False, False)
    bf.assign()
    return bf
# Round robin
def rr_wrapper(N: int, load_vectors):
    return RoundRobin(N, load_vectors, False)

algorithms = [
    ('SALP', salp_wrapper),
    ('BF', bf_wrapper),
    ('RR', rr_wrapper),
]

# Parametry dla iteracji
* F - ilość shardów
* N - ilość węzłów
* cor - poziom korelacji miedzy wektorami obciążenia
* ro - poziom obciążenia chmury
* size - długość wektorów obciążenia
* repeats - ilość powtórzeń pojedynczej symulacji
* parametry generatora do ustalenia

In [3]:
N = 100
F = 10*N
cor_range = np.arange(0.0, 1.01, 0.5)
load_range = [0.8, 0.9]
size=100
repeats=5
rng = (2,4)

In [4]:
def gamma_generator_factory(cor):
    return ga.Generator(F, size, cor, rng)

In [5]:
import pandas as pd

gen = pipeline(N, size, repeats, cor_range, load_range, gamma_generator_factory, algorithms)
df = pd.DataFrame(gen, columns=['correlation', 'load', 'algorithm', 'value'])

df.head()

100%|██████████| 30/30 [00:52<00:00,  1.74s/it]


,correlation,load,algorithm,value
0,0.0,0.8,SALP,2.303014e+06
1,0.0,0.8,BF,2.307079e+06
2,0.0,0.8,RR,2.524497e+06
3,0.0,0.8,SALP,2.275835e+06
4,0.0,0.8,BF,2.280970e+06
